In [1]:
import joblib
import os
from numpy.linalg import inv
from API.utils import *
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

In [2]:
os.listdir('./model')

['ANN(58_to_sp162(二合一)).pkl',
 'ANN(58_to_sp162).pkl',
 'ANN(58_to_y162(二合一)).pkl',
 'ANN(58_to_y162).pkl',
 'ANN(energy).pkl',
 'ANN(tray).pkl',
 'eva.pkl',
 'OperationOptimModel.pkl',
 'SVR(4_to_23).pkl',
 'transformer(23_to_54).pkl',
 'transformer(33_to_15).pkl',
 'transformer(43_to_33).pkl',
 'transformer(54_to_23).pkl',
 'transformer(54_to_33).pkl',
 'transformer(SP162_to_Y162).pkl']

In [3]:
os.listdir('./data/phase_2/cleaned')

['col_names.pkl', 'train(energy).csv', 'train.csv']

# load pretrained model

In [4]:
A = joblib.load('./model/SVR(4_to_23).pkl')
B = joblib.load('./model/transformer(23_to_54).pkl')
C = joblib.load('./model/ANN(58_to_y162(二合一)).pkl')
#D = joblib.load('./model/transformer(SP162_to_Y162).pkl')
E = joblib.load('./model/ANN(energy).pkl')
F = joblib.load('./model/transformer(54_to_33).pkl')
G = joblib.load('./model/transformer(33_to_15).pkl')
H = joblib.load('./model/transformer(43_to_33).pkl')
I = joblib.load('./model/transformer(54_to_23).pkl')
col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')

In [5]:
col_names.keys()

dict_keys(['xna', 'xle', 'xhc', 'xhe', 'fna', 'fle', 'fhc', 'fhe', 'yld', 'Tray_temp', 'duty', 'Rate_m3', 'Rate_ton', 'sle', 'shc', 'she', 'case'])

# A

In [6]:
A_X = pd.read_csv('./data/phase_1/test_170.csv',index_col=0).sample(1)[['T10','T50','T90','N+A']]
A_X.head()

,T10,T50,T90,N+A
18,106.8,124.1,149.3,32.995


In [7]:
A_pred = A.predict(A_X)
print(A_pred.sum(axis=1))
A_pred

0    100.0
dtype: float64


,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,0.147128,0.11434,0.103852,2.823517,0.907467,2.019083,0.377365,10.109991,7.250203,6.193628,2.897328,9.580959,10.082132,5.086986,5.972155,7.016073,11.041471,4.404671,4.399328,1.352728,6.578992,0.619084,0.921521


# B

In [8]:
B_pred = B(A_pred)
print(B_pred.sum(axis=1))
B_pred.head()

0    100.0
dtype: float64


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,0.0,0.0,0.0,0.005717,0.011434,0.094902,0.147128,0.002287,0.103852,0.889317,0.018149,2.823517,1.309893,0.694474,0.377365,0.709189,0.763082,0.015222,5.777425,0.573316,10.109991,0.0,3.321019,2.299292,0.100821,2.897328,9.98131,9.580959,5.086986,1.418118,0.804446,1.985576,1.764015,10.760988,0.280482,7.016073,0.062353,2.642682,1.761989,0.265962,1.142163,1.687989,1.240861,1.106346,1.641542,3.831103,0.619084,1.13436,0.861714,0.214543,0.050098,0.003825,0.009709,0.0


# B2 54 to 23

In [9]:
B.W.shape

(23, 54)

# C

In [10]:
case = pd.read_csv('./data/phase_2/cleaned/train.csv',index_col=0)[col_names['case']].sample(1).reset_index(drop=True)
C_X = case.join(B_pred)
C_X

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,20.0,2.5,100.0,2.0,0.0,0.0,0.0,0.005717,0.011434,0.094902,0.147128,0.002287,0.103852,0.889317,0.018149,2.823517,1.309893,0.694474,0.377365,0.709189,0.763082,0.015222,5.777425,0.573316,10.109991,0.0,3.321019,2.299292,0.100821,2.897328,9.98131,9.580959,5.086986,1.418118,0.804446,1.985576,1.764015,10.760988,0.280482,7.016073,0.062353,2.642682,1.761989,0.265962,1.142163,1.687989,1.240861,1.106346,1.641542,3.831103,0.619084,1.13436,0.861714,0.214543,0.050098,0.003825,0.009709,0.0


In [11]:
D_pred,C_pred = C.predict(C_X)
print(C_pred.sum(axis=1))
C_pred

0    53.999878
dtype: float32


,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heavy End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heavy End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heavy End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heavy End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Heavy End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Heart Cut Split Factor_i-Pentane_Ratio,Individual Component to Heavy End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Heart Cut Split Factor_n-Pentane_Ratio,Individual Component to Heavy End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Pentene_Ratio,Individual Component to Heavy End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Cyclopentane_Ratio,Individual Component to Heavy End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Heart Cut Split Factor_3-Methylpentane_Ratio,Individual Component to Heavy End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Hexene_Ratio,Individual Component to Heavy End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Heart Cut Split Factor_n-Hexane_Ratio,Individual Component to Heavy End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Methylcyclopentane_Ratio,Individual Component to Heavy End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heart Cut Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heavy End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Heart Cut Split Factor_Benzene_Ratio,Individual Component to Heavy End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,Individual Component to Heart Cut Split Factor_Cyclohexane_Ratio,Individual Component to Heavy End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heart Cut Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heavy End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heart Cut Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heavy End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,Individual Component to Heart Cut Split Factor_2-Methylhexane_Ratio,Individual Component to Heavy End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heart Cut Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heavy End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Compone

# D

In [12]:
print(D_pred.sum(axis=1))
D_pred

0    300.0
dtype: float32


,Light End Product Properties_Oxygen_wt%,Light End Product Properties_Methane_wt%,Light End Product Properties_Ethane_wt%,Light End Product Properties_Propane_wt%,Light End Product Properties_n-Butane_wt%,Light End Product Properties_i-Pentane_wt%,Light End Product Properties_n-Pentane_wt%,Light End Product Properties_tr2-Pentene_wt%,Light End Product Properties_Cyclopentane_wt%,Light End Product Properties_3-Methylpentane_wt%,Light End Product Properties_tr2-Hexene_wt%,Light End Product Properties_n-Hexane_wt%,Light End Product Properties_Methylcyclopentane_wt%,"Light End Product Properties_2,2-Dimethylpentane_wt%",Light End Product Properties_Benzene_wt%,Light End Product Properties_Cyclohexane_wt%,"Light End Product Properties_2,2,3-Trimethylbutane_wt%",Light End Product Properties_4-Methyl-1-hexene_wt%,Light End Product Properties_2-Methylhexane_wt%,"Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%",Light End Product Properties_n-Heptane_wt%,Light End Product Properties_H2O_wt%,Light End Product Properties_Methylcyclohexane_wt%,Light End Product Properties_Ethylcyclopentane_wt%,"Light End Product Properties_2,3-Dimethyl-1-hexene_wt%",Light End Product Properties_Toluene_wt%,Light End Product Properties_2-Methylheptane_wt%,Light End Product Properties_n-Octane_wt%,Light End Product Properties_Ethylcyclohexane_wt%,Light End Product Properties_EthylBenzene_wt%,Light End Product Properties_P-Xylene_wt%,Light End Product Properties_M-Xylene_wt%,Light End Product Properties_O-Xylene_wt%,Light End Product Properties_2-Methyloctane_wt%,Light End Product Properties_1-Nonene_wt%,Light End Product Properties_n-Nonane_wt%,Light End Product Properties_i-Propylbenzene_wt%,"Light End Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Light End Product Properties_n-Propylcyclohexane_wt%,Light End Product Properties_n-Propylbenzene_wt%,"Light End Product Properties_1,3,5-Trimethylbenzene_wt%",Light End Product Properties_1-Methyl-2-ethylbenzene_wt%,"Light End Product Properties_1,2,3-Trimethylbenzene_wt%","Light End Product Properties_2,4-Dimethyloctane_wt%","Light End Product Properties_2,2,3,3-Tetramethylhexane_wt%",Light End Product Properties_3-Methylnonane_wt%,Light End Product Properties_tert-Butylcyclohexane_wt%,Light End Product Properties_n-Decane_wt%,"Light End Product Properties_1,2-Diethylbenzene_wt%",Light End Product Properties_n-Undecane_wt%,Light End Product Properties_n-Pentylbenzene_wt%,Light End Product Properties_n-Dodecane_wt%,Light End Product Properties_n-Hexylbenzene_wt%,Light End Product Properties_n-Tridecane_wt%,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product

# E

In [13]:
df = pd.read_csv('./data/phase_2/cleaned/train(energy).csv',index_col=0).dropna(axis=0).sample(1)
E_X = df[E.x_col]
E_X

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%,Naphtha Properties_Oxygen_wt%,Naphtha Properties_Methane_wt%,Naphtha Properties_Ethane_wt%,Naphtha Properties_Propane_wt%,Naphtha Properties_n-Butane_wt%,Naphtha Properties_i-Pentane_wt%,Naphtha Properties_n-Pentane_wt%,Naphtha Properties_tr2-Pentene_wt%,Naphtha Properties_Cyclopentane_wt%,Naphtha Properties_3-Methylpentane_wt%,Naphtha Properties_tr2-Hexene_wt%,Naphtha Properties_n-Hexane_wt%,Naphtha Properties_Methylcyclopentane_wt%,"Naphtha Properties_2,2-Dimethylpentane_wt%",Naphtha Properties_Benzene_wt%,Naphtha Properties_Cyclohexane_wt%,"Naphtha Properties_2,2,3-Trimethylbutane_wt%",Naphtha Properties_4-Methyl-1-hexene_wt%,Naphtha Properties_2-Methylhexane_wt%,"Naphtha Properties_t-1,3-Dimethylcyclopentane_wt%",Naphtha Properties_n-Heptane_wt%,Naphtha Properties_H2O_wt%,Naphtha Properties_Methylcyclohexane_wt%,Naphtha Properties_Ethylcyclopentane_wt%,"Naphtha Properties_2,3-Dimethyl-1-hexene_wt%",Naphtha Properties_Toluene_wt%,Naphtha Properties_2-Methylheptane_wt%,Naphtha Properties_n-Octane_wt%,Naphtha Properties_Ethylcyclohexane_wt%,Naphtha Properties_EthylBenzene_wt%,Naphtha Properties_P-Xylene_wt%,Naphtha Properties_M-Xylene_wt%,Naphtha Properties_O-Xylene_wt%,Naphtha Properties_2-Methyloctane_wt%,Naphtha Properties_1-Nonene_wt%,Naphtha Properties_n-Nonane_wt%,Naphtha Properties_i-Propylbenzene_wt%,"Naphtha Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Naphtha Properties_n-Propylcyclohexane_wt%,Naphtha Properties_n-Propylbenzene_wt%,"Naphtha Properties_1,3,5-Trimethylbenzene_wt%",Naphtha Properties_1-Methyl-2-ethylbenzene_wt%,"Naphtha Properties_1,2,3-Trimethylbenzene_wt%","Naphtha Properties_2,4-Dimethyloctane_wt%","Naphtha Properties_2,2,3,3-Tetramethylhexane_wt%",Naphtha Properties_3-Methylnonane_wt%,Naphtha Properties_tert-Butylcyclohexane_wt%,Naphtha Properties_n-Decane_wt%,"Naphtha Properties_1,2-Diethylbenzene_wt%",Naphtha Properties_n-Undecane_wt%,Naphtha Properties_n-Pentylbenzene_wt%,Naphtha Properties_n-Dodecane_wt%,Naphtha Properties_n-Hexylbenzene_wt%,Naphtha Properties_n-Tridecane_wt%
396-21,30.0,10.0,100.0,1.0,0.0,0.0,0.0,0.014,0.028001,0.231005,0.397008,0.006,0.031001,0.451009,0.009,0.908018,3.437069,0.784016,0.160003,1.861037,0.861017,0.017,6.521131,2.043041,7.274145,0.0,11.835237,8.194163,0.068001,5.244105,6.778135,5.334107,9.333186,1.194024,0.677014,1.672033,1.48503,6.749135,0.176004,2.678053,0.031001,3.359067,2.239045,0.133003,0.573011,0.847017,0.622012,0.615012,0.912018,2.129042,0.815016,0.465009,0.671013,0.088002,0.039001,0.002,0.008,0.0


In [14]:
E_pred = E.predict(E_X)
E_pred

,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Reboiler Heat Duty_Mkcal/hr,NA_density,LE_density,HC_density,HE_density
0,2.154692,1.962225,0.752115,0.700892,0.751822,0.774564


# F

In [15]:
F_X = D_pred[col_names['xhc']]
F_X

,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product Properties_P-Xylene_wt%,Heart Cut Product Properties_M-Xylene_wt%,Heart Cut Product Properties_O-Xylene_wt%,Heart Cut Product Properties_2-Methyloctane_wt%,Heart Cut Product Properties_1-Nonene_wt%,Heart Cut Product Properties_n-Nonane_wt%,Heart Cut Product Properties_i-Propylbenzene_wt%,"Heart Cut Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Heart Cut Product Properties_n-Propylcyclohexane_wt%,Heart Cut Product Properties_n-Propylbenzene_wt%,"Heart Cut Product Properties_1,3,5-Trimethylbenzene_wt%",Heart Cut Product Properties_1-Methyl-2-ethylbenzene_wt%,"Heart Cut Product Properties_1,2,3-Trimethylbenzene_wt%","Heart Cut Product Properties_2,4-Dimethyloctane_wt%","Heart Cut Product Properties_2,2,3,3-Tetramethylhexane_wt%",Heart Cut Product Properties_3-Methylnonane_wt%,Heart Cut Product Properties_tert-Butylcyclohexane_wt%,Heart Cut Product Properties_n-Decane_wt%,"Heart Cut Product Properties_1,2-Diethylbenzene_wt%",Heart Cut Product Properties_n-Undecane_wt%,Heart Cut Product Properties_n-Pentylbenzene_wt%,Heart Cut Product Properties_n-Dodecane_wt%,Heart Cut Product Properties_n-Hexylbenzene_wt%,Heart Cut Product Properties_n-Tridecane_wt%
0,0.0,0.0,0.0,0.000002,0.000007,0.000476,0.00217,0.000031,0.009994,0.337243,0.010828,1.649243,0.921253,0.721907,0.275898,0.759808,0.850902,0.017659,6.701877,0.665893,11.745026,0.0,3.857508,2.670298,0.117084,3.365195,11.587005,11.058762,5.545841,1.541244,0.830154,2.021835,1.699083,9.872894,0.244166,5.977393,0.054183,2.486758,1.455198,0.215365,0.894754,1.396767,0.968741,0.904642,1.332379,3.020284,0.490186,0.868762,0.655476,0.151328,0.038175,0.002706,0.005611,0.0


In [16]:
F_pred = F(F_X)
print(F_pred.sum(axis=1))
F_pred

0    99.999995
dtype: float64


,Hydrogen,Methane,Ethane,Propane,n-Butane,i-Butane,C4 Naphthenes,n-Pentane,C5 i-Paraffins,C5 Naphthenes,n-Hexane,C6 i-Paraffins,C6 Naphthenes,Benzene,n-Heptane,C7 i-Paraffins,C7 Naphthenes,Toluene,n-Octane,C8 i-Paraffins,C8 Naphthenes,Ethylbenzene,P-Xylene,M-Xylene,O-Xylene,n-Nonane,C9 i-Paraffins,C9 Naphthenes,C9 Aromtics,C10+ n-Paraffins,C10+ i-Paraffins,C10+ Naphthenes,C10+ Aromtics
0,0.0,0.0,0.0,0.000002,0.000003,0.000004,3.486844e-07,0.00217,0.000507,0.009994,1.649243,0.348071,1.681062,0.275898,11.745026,8.292344,7.193699,3.365195,11.058762,11.704089,5.545841,1.541244,0.830154,2.021835,1.699083,5.977393,10.11706,3.941955,3.529811,1.022797,5.257305,0.490186,0.699261


# G

In [17]:
G_pred = G(F_pred)
print(G_pred.sum(axis=1))
G_pred

0    99.999995
dtype: float64


,HYD,NC1,NC2,NC3,PC4,SAP,SAN,BEZ,TOL,ETB,PXY,MXY,OXY,C9A,C10
0,0.0,0.0,0.0,0.000002,0.000007,67.174766,18.862738,0.275898,3.365195,1.541244,0.830154,2.021835,1.699083,3.529811,0.699261


# H

In [18]:
x = np.random.uniform(size=(1,len(H.x_col)))
x = pd.DataFrame(x,columns=H.x_col)
x

,"N+2A in Naphtha Feed, wt%","Chlorine in Fresh Catalyst, wt%","Carbon in Spent Catalyst, %","C6P- in Naphtha Feed, wt%","3R211 Inlet Tem, oC","3R212 Inlet Temp, oC","3R213 Inlet Temp, oC","3R214 Inlet Temp, oC",Feed Hydrogen,Feed Methane,Feed Ethane,Feed Propane,Feed n-Butane,Feed i-Butane,Feed C4 Naphthenes,Feed n-Pentane,Feed C5 i-Paraffins,Feed C5 Naphthenes,Feed n-Hexane,Feed C6 i-Paraffins,Feed C6 Naphthenes,Feed Benzene,Feed n-Heptane,Feed C7 i-Paraffins,Feed C7 Naphthenes,Feed Toluene,Feed n-Octane,Feed C8 i-Paraffins,Feed C8 Naphthenes,Feed Ethylbenzene,Feed P-Xylene,Feed M-Xylene,Feed O-Xylene,Feed n-Nonane,Feed C9 i-Paraffins,Feed C9 Naphthenes,Feed C9 Aromtics,Feed C10+ n-Paraffins,Feed C10+ i-Paraffins,Feed C10+ Naphthenes,Feed C10+ Aromtics,"Product Separator Pressure, kg/cm2_g",H2/HC Ratio
0,0.101062,0.720584,0.573636,0.046712,0.713966,0.640194,0.726203,0.499962,0.160045,0.223498,0.200329,0.504294,0.023428,0.343213,0.46156,0.350126,0.570228,0.424176,0.435157,0.038587,0.045201,0.998307,0.282663,0.875229,0.549413,0.115914,0.571033,0.381335,0.268285,0.348071,0.756755,0.530716,0.30159,0.078523,0.832904,0.002354,0.41841,0.720584,0.898383,0.47264,0.492219,0.640267,0.064172


In [19]:
H.predict(x)

,Product Hydrogen,Product Methane,Product Ethane,Product Propane,Product n-Butane,Product i-Butane,Product C4 Naphthenes,Product n-Pentane,Product C5 i-Paraffins,Product C5 Naphthenes,Product n-Hexane,Product C6 i-Paraffins,Product C6 Naphthenes,Product Benzene,Product n-Heptane,Product C7 i-Paraffins,Product C7 Naphthenes,Product Toluene,Product n-Octane,Product C8 i-Paraffins,Product C8 Naphthenes,Product Ethylbenzene,Product P-Xylene,Product M-Xylene,Product O-Xylene,Product n-Nonane,Product C9 i-Paraffins,Product C9 Naphthenes,Product C9 Aromtics,Product C10+ n-Paraffins,Product C10+ i-Paraffins,Product C10+ Naphthenes,Product C10+ Aromtics
0,0.682839,0.137013,0.783949,0.326594,0.105715,0.091932,0.0,0.211992,0.333332,0.001654,0.106522,0.262807,0.010453,0.290096,0.308836,1.083908,0.015893,2.34887,0.070044,0.06163,0.014382,0.598901,0.375767,2.530017,0.928525,0.000723,0.003143,0.000266,1.769434,0.001956,0.005107,0.003207,0.209664


# I

In [20]:
A_pred = A.predict(A_X)
print(A_pred.sum(axis=1))
A_pred

0    100.0
dtype: float64


,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,0.147128,0.11434,0.103852,2.823517,0.907467,2.019083,0.377365,10.109991,7.250203,6.193628,2.897328,9.580959,10.082132,5.086986,5.972155,7.016073,11.041471,4.404671,4.399328,1.352728,6.578992,0.619084,0.921521


In [21]:
B_pred = B(A_pred)
print(B_pred.sum(axis=1))
B_pred.head()

0    100.0
dtype: float64


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,0.0,0.0,0.0,0.005717,0.011434,0.094902,0.147128,0.002287,0.103852,0.889317,0.018149,2.823517,1.309893,0.694474,0.377365,0.709189,0.763082,0.015222,5.777425,0.573316,10.109991,0.0,3.321019,2.299292,0.100821,2.897328,9.98131,9.580959,5.086986,1.418118,0.804446,1.985576,1.764015,10.760988,0.280482,7.016073,0.062353,2.642682,1.761989,0.265962,1.142163,1.687989,1.240861,1.106346,1.641542,3.831103,0.619084,1.13436,0.861714,0.214543,0.050098,0.003825,0.009709,0.0


In [22]:
I(B_pred)

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,0.147128,0.11434,0.103852,2.823517,0.907467,2.019083,0.377365,10.109991,7.250203,6.193628,2.897328,9.580959,10.082132,5.086986,5.972155,7.016073,11.041471,4.404671,4.399328,1.352728,6.578992,0.619084,0.921521


# 把全部模組整合到一塊

In [23]:
class EVA(object):
    def __init__(self):
        self.A = joblib.load('./model/SVR(4_to_23).pkl')
        self.B = joblib.load('./model/transformer(23_to_54).pkl')
        self.C = joblib.load('./model/ANN(58_to_y162(二合一)).pkl')# 預測wt%同時給出分離係數
        #self.D = joblib.load('./model/transformer(SP162_to_Y162).pkl') #不需要了
        self.E = joblib.load('./model/ANN(energy).pkl')
        self.F = joblib.load('./model/transformer(54_to_33).pkl')
        self.G = joblib.load('./model/transformer(33_to_15).pkl')
        self.H = joblib.load('./model/transformer(43_to_33).pkl')
        self.I = joblib.load('./model/transformer(54_to_23).pkl')
        
        self.col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')

    def __call__(self, X):
        x4, case4 = X.iloc[:, :4], X.iloc[:, 4:]
        # model forward
        self.y23 = self.A.predict(x4)  # percent sum = 100
        self.Xna = self.B(self.y23)  # percent sum = 100
        # percent sum = 1 * 54component = 54
        self.y162,self.sp162 = self.C.predict(case4.join(self.Xna))
        
        self.xhc = self.y162[self.col_names['xhc']]  # percent sum = 100
        self.xhc33_p = self.F(self.xhc)/100  # percent

        # predict duty and density(ton/m3)
        E_pred = self.E.predict(case4.join(self.Xna))
        self.duty = E_pred.iloc[:, :2]
        self.density = E_pred.iloc[:, 2:]

        # get split factor
        self.shc = self.sp162[self.col_names['shc']]
        self.sle = self.sp162[self.col_names['sle']]
        self.she = self.sp162[self.col_names['she']]

        # fhc_ton = fhc_m3*density
        self.fhc_m3 = case4['Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr']
        self.fhc_ton = self.fhc_m3.values * self.density.iloc[:, 2].values

        # calculate fle_ton,fhe_ton
        self.fle_ton = self.fhc_ton * ((self.Xna.values@self.sle.values.ravel()) /
                                       (self.Xna.values@self.shc.values.ravel()))
        self.fhe_ton = self.fhc_ton * ((self.Xna.values@self.she.values.ravel()) /
                                       (self.Xna.values@self.shc.values.ravel()))

        # calculate fna
        self.fna_ton = self.fle_ton + self.fhc_ton + self.fhe_ton

        # scale fle,fhc,fhe
        self.fle_ton = self.fle_ton*100000/self.fna_ton
        self.fhc_ton = self.fhc_ton*100000/self.fna_ton
        self.fhe_ton = self.fhe_ton*100000/self.fna_ton

        # add columns name
        self.fle_ton = pd.Series(
            self.fle_ton, name=self.col_names['Rate_ton'][1])
        self.fhe_ton = pd.Series(
            self.fhe_ton, name=self.col_names['Rate_ton'][3])

        # scale duty
        self.duty *= 100/self.fna_ton[0]

        # xhc33 = fhc_ton*xhc_33_p
        self.xhc33 = self.fhc_ton[0]*self.xhc33_p
        self.xhc33_tmp = self.xhc33*74000/self.fhc_ton[0]

        # xhe33 = H(someting + xhc33 + something)
        xhc23 = self.I(self.xhc)
        C6Pm = xhc23[['C5NP', 'C5IP', 'C6IP', 'C6NP']].sum(axis=1)[0]

        NpA = xhc23[['C5N', 'C6N', 'C7N', 'C8N', 'C9N', 'C10N']].sum(axis=1)[0] +\
            xhc23[['C6A', 'C7A', 'C8A', 'C9A', 'C10A']].sum(axis=1)[0]*2

        self.H_input = [NpA, 0.942, 3.78, C6Pm, 517, 517, 517, 517] + \
            self.xhc33_tmp.values.ravel().tolist()+[4.798, 1.44]
        self.H_input = np.array([self.H_input])
        self.H_input = pd.DataFrame(self.H_input, columns=self.H.x_col)
        self.reformer_input = [NpA, 0.942, 3.78, C6Pm, 517, 517, 517, 517] + self.xhc33.values.ravel().tolist() + [4.798, 1.44]
        self.reformer_input = np.array([self.reformer_input])
        self.reformer_input = pd.DataFrame(self.reformer_input, columns=self.H.x_col)

        self.重組33_tmp = self.H.predict(self.H_input)

        self.重組33 = self.重組33_tmp *self.fhc_ton[0]/74000

        # final output
        self.重組15 = self.G(self.重組33)

        # raname and combine return to user
        self.predict = self.重組15.join(self.fle_ton).join(
            self.fhe_ton).join(self.duty)
        self.naphtha = self.y23
        self.pre_d = self.sp162
        self.reform = self.重組33

In [24]:
eva = EVA()

# save eva

In [25]:
joblib.dump(eva,'./model/eva.pkl')
del eva

# load eva

In [26]:
eva = joblib.load('./model/eva.pkl')

# create input X for eva

In [27]:
X = pd.DataFrame(np.hstack((A_X.values,case.values)))
X.columns = A_X.columns.tolist()+case.columns.tolist()
X.head()

,T10,T50,T90,N+A,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%
0,106.8,124.1,149.3,32.995,20.0,2.5,100.0,2.0


# get eva result

In [28]:
eva(X)

In [29]:
eva.predict

,HYD,NC1,NC2,NC3,PC4,SAP,SAN,BEZ,TOL,ETB,PXY,MXY,OXY,C9A,C10,Light End Product Properties_Light End Prod. Rate_ton/hr,Heavy End Product Properties_Heavy End Product Rate_ton/hr,Duty_E062 3.5K Steam Rate_ton/hr,Duty_Reboiler Heat Duty_Mkcal/hr
0,2833.976216,815.952378,3779.603459,3451.782054,2396.116973,16483.350497,766.70624,1903.602378,14767.491459,3728.716108,2913.995459,10604.755027,5546.586811,12608.166054,3186.655784,3478.657597,10733.878529,2.583787,2.862675


In [30]:
eva.xhc33

,Hydrogen,Methane,Ethane,Propane,n-Butane,i-Butane,C4 Naphthenes,n-Pentane,C5 i-Paraffins,C5 Naphthenes,n-Hexane,C6 i-Paraffins,C6 Naphthenes,Benzene,n-Heptane,C7 i-Paraffins,C7 Naphthenes,Toluene,n-Octane,C8 i-Paraffins,C8 Naphthenes,Ethylbenzene,P-Xylene,M-Xylene,O-Xylene,n-Nonane,C9 i-Paraffins,C9 Naphthenes,C9 Aromtics,C10+ n-Paraffins,C10+ i-Paraffins,C10+ Naphthenes,C10+ Aromtics
0,0.0,0.0,0.0,0.001868,0.002393,0.00329,0.000299,1.86146,0.434879,8.573868,1414.843638,298.601344,1442.140186,236.686084,10075.759623,7113.791968,6171.292137,2886.91568,9487.03111,10040.641098,4757.636122,1322.194268,712.167911,1734.480842,1457.60019,5127.85358,8679.169261,3381.703503,3028.135569,877.431294,4510.108803,420.518526,599.878663


In [31]:
eva.naphtha

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,0.147128,0.11434,0.103852,2.823517,0.907467,2.019083,0.377365,10.109991,7.250203,6.193628,2.897328,9.580959,10.082132,5.086986,5.972155,7.016073,11.041471,4.404671,4.399328,1.352728,6.578992,0.619084,0.921521


In [32]:
eva.pre_d

,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heavy End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heavy End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heavy End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heavy End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Heavy End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Heart Cut Split Factor_i-Pentane_Ratio,Individual Component to Heavy End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Heart Cut Split Factor_n-Pentane_Ratio,Individual Component to Heavy End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Pentene_Ratio,Individual Component to Heavy End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Cyclopentane_Ratio,Individual Component to Heavy End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Heart Cut Split Factor_3-Methylpentane_Ratio,Individual Component to Heavy End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Heart Cut Split Factor_tr2-Hexene_Ratio,Individual Component to Heavy End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Heart Cut Split Factor_n-Hexane_Ratio,Individual Component to Heavy End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,Individual Component to Heart Cut Split Factor_Methylcyclopentane_Ratio,Individual Component to Heavy End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heart Cut Split Factor_2,2-Dimethylpentane_Ratio","Individual Component to Heavy End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Heart Cut Split Factor_Benzene_Ratio,Individual Component to Heavy End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,Individual Component to Heart Cut Split Factor_Cyclohexane_Ratio,Individual Component to Heavy End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heart Cut Split Factor_2,2,3-Trimethylbutane_Ratio","Individual Component to Heavy End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heart Cut Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Heavy End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,Individual Component to Heart Cut Split Factor_2-Methylhexane_Ratio,Individual Component to Heavy End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heart Cut Split Factor_t-1,3-Dimethylcyclopentane_Ratio","Individual Component to Heavy End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Compone

In [33]:
eva.reform

,Product Hydrogen,Product Methane,Product Ethane,Product Propane,Product n-Butane,Product i-Butane,Product C4 Naphthenes,Product n-Pentane,Product C5 i-Paraffins,Product C5 Naphthenes,Product n-Hexane,Product C6 i-Paraffins,Product C6 Naphthenes,Product Benzene,Product n-Heptane,Product C7 i-Paraffins,Product C7 Naphthenes,Product Toluene,Product n-Octane,Product C8 i-Paraffins,Product C8 Naphthenes,Product Ethylbenzene,Product P-Xylene,Product M-Xylene,Product O-Xylene,Product n-Nonane,Product C9 i-Paraffins,Product C9 Naphthenes,Product C9 Aromtics,Product C10+ n-Paraffins,Product C10+ i-Paraffins,Product C10+ Naphthenes,Product C10+ Aromtics
0,2833.976216,815.952378,3779.603459,3451.782054,1284.570919,1111.546054,0.815223,2599.878486,3937.664,25.497019,631.477514,1544.624757,80.449372,1903.602378,1288.684865,5212.340757,274.038486,14767.491459,171.414351,607.354973,150.472878,3728.716108,2913.995459,10604.755027,5546.586811,9.135734,34.632905,22.453525,12608.166054,149.1105,296.216432,213.794959,3186.655784
